In [1]:
import os
import numpy as np
import torch
import torch.nn.functional as F
from transformers import GPT2Tokenizer, TrainingArguments
from utils.helper import load_model_recursive
from ppcm_models.pytorch_pretrained_bert.modeling_adapter import GPT2LMHeadModel, GPT2Config

In [2]:
class DataArguments():
    def __init__(self):
        self.dataset_path = '/home/bryan/datasets/bookcorpusopen/bookcorpusopen_chunked.arrow'
        self.bookcorpusopen_story_column_name = 'chunk'
        self.preprocessing_num_workers = 8
        self.genre='Romance'
        self.adapter_id=1
        self.match_up_to_n_genres=3
        self.sample_row=None
        
class ModelArguments():
    def __init__(self):
        self.model_size = 'small'
        self.load_checkpoint_adapter = ""
        self.max_seq_len=512
        # self.lr = 2e-4 #, help="Learning rate")

class TrainingArguments(TrainingArguments):
    def __init__(self):
        self.output_dir = "./save"
        self.eval_accumulation_steps = None
        
model_args = ModelArguments()
data_args = DataArguments()
training_args = TrainingArguments()

In [3]:
model_args.model_path = f'ppcm_models/dialoGPT/small/'
config = GPT2Config.from_json_file(os.path.join(model_args.model_path, 'config.json'))

tokenizer = GPT2Tokenizer.from_pretrained(model_args.model_path)

In [4]:
model_run_names = ['GPT2small_adapterid0_genreAction_matched3_sampleNone_maxseqlen512_bs8_lr5e-05_10.0epoch_wd0.0_ws0',
                   'GPT2small_adapterid0_genreAction_matched3_sampleNone_maxseqlen512_bs8_lr5e-05_2.0epoch_wd0.0_ws0',
                   'GPT2small_adapterid0_genreMystery_matched3_sampleNone_maxseqlen512_bs8_lr0.0005_5.0epoch_wd0.0_ws0',
                   'GPT2small_adapterid0_genreRomance_matched3_sampleNone_maxseqlen512_bs8_lr5e-05_2.0epoch_wd0.0_ws0']

for i, model_run_name in enumerate(model_run_names):

    path = './save/'+model_run_name+'/pytorch_model.bin'
    model = load_model_recursive(GPT2LMHeadModel(config), path, model_args, verbose=True)

    # # use this to generate outputs
    # inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
    # outputs = model(inputs['input_ids'], task_id=[0])
    # # or
    # outputs = model.transformer(inputs['input_ids'], task_id=0)

    length = 100
    text = "Hello, my dog is cute"
    generated = tokenizer.encode(text)
    context = torch.tensor([generated])

    generation = model.generate(inputs=context,
                               num_beams=3, 
                               length_penalty=3, 
                               early_stopping=1, 
                               num_beam_groups=3, 
                               do_sample=False, 
                               num_return_sequences=2, 
                               bos_token_id=50256,
                               eos_token_id=50256,
                               pad_token_id=50256,
                               output_scores=True,
                               output_attentions=True,
                               output_hidden_states=True,
                               return_dict_in_generate=True,
                               repetition_penalty=1.1,
                               min_length = 0,
                               max_length = length,
                               no_repeat_ngram_size=2,
                               encoder_no_repeat_ngram_size=False,
                               bad_words_ids=[[100]], # tokenizer.decode(100)
                               diversity_penalty=0.2,
                               forced_bos_token_id=50256,
                               forced_eos_token_id=50256,
                               remove_invalid_values=True,
                               exponential_decay_length_penalty=[1.0, 1.2])

    print(tokenizer.decode(generation[0][0]), '\n')

Loading finetuned model from ./save/GPT2small_adapterid0_genreAction_matched3_sampleNone_maxseqlen512_bs8_lr5e-05_10.0epoch_wd0.0_ws0/pytorch_model.bin


/home/bryan/miniconda3/envs/pix2story/lib/python3.10/site-packages/torch/cuda/__init__.py:145: UserWarning: 
GeForce RTX 3090 with CUDA capability sm_86 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_70.
If you want to use the GeForce RTX 3090 GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))
/home/bryan/miniconda3/envs/pix2story/lib/python3.10/site-packages/transformers/generation_beam_search.py:196: UserWarning: Passing `max_length` to BeamSearchScorer is deprecated and has no effect. `max_length` should be passed directly to `beam_search(...)`, `beam_sample(...)`, or `group_beam_search(...)`.
  warnings.warn(


Hello, my dog is cute. I'm going to take her to the vet. She's a little bit scared. But I'll be okay.""I'm sorry, but I don't know what to do. It's not like I want to be in a relationship with you. You're my best friend, and I love you."The man was a man of few words. He was tall, thin, with a thick, dark complexion. His hair was long and flowing. The<|endoftext|> 

Loading finetuned model from ./save/GPT2small_adapterid0_genreAction_matched3_sampleNone_maxseqlen512_bs8_lr5e-05_2.0epoch_wd0.0_ws0/pytorch_model.bin
Hello, my dog is cute. I'm not sure if she's cute or not. But I think she is. She's a little bit cute, but I don't think I can tell you what she looks like."I'm sorry, I was just trying to get you to stop talking about me. It's not like I've ever talked to you before. You're a good guy. And I know you're not.""You're right. We're in the middle of<|endoftext|> 

Loading finetuned model from ./save/GPT2small_adapterid0_genreMystery_matched3_sampleNone_maxseqlen512_bs8_lr0.0005_